# Lecture 07. 모아보기 데이터 수집
### 목표: 포스트 목록(20개)에서 모아보기에서 필요한 데이터를 수집한다.

## 포스트 목록(20개)에 대한 데이터 수집
**미션**: 
1. for-루프를 사용해서 모든 포스트에 대한 데이터를 수집한다.
2. 포스트 주소가 존재하는지 확인하는 코드를 추가한다.

In [ ]:
import requests
from bs4 import BeautifulSoup

from datetime import datetime
from datetime import timedelta

import moabogey_database as moabogey
from moabogey_id import *

site_url = 'https://twitter.com/BTS_ARMY'
response = requests.get(site_url)
print('status code: ', response.status_code)

if(response.status_code == requests.codes.ok):
    html_source = response.text
    soup = BeautifulSoup(html_source, 'html.parser')
    
    db_name = 'lecture07'
    my_db = moabogey.Dbase(db_name, bot_id)
    
    for post in soup.find_all('div', class_='content'):
        href = post.find('a', {"class": "tweet-timestamp", "href": True})
        if href:
            href_full = 'https://twitter.com' + href['href']            
            createdBy = post.find('strong', class_='fullname').text.strip()  
            post_time = post.find('a', class_="tweet-timestamp")['title']
            post_time = post_time.replace('오전', 'AM').replace('오후', 'PM')
            createdAt = datetime.strptime(post_time, '%p %I:%M - %Y년 %m월 %d일')

            post_url = href_full
            res = requests.get(post_url)

            if(res.status_code == requests.codes.ok):
                post_source = res.text
                post = BeautifulSoup(post_source, 'html.parser')

                title = post.find('meta', property="og:description")
                title = title['content'].splitlines()[0]
                desc = post.find('meta', property="og:description")
                desc = ''.join(desc['content'].splitlines()[1:])
                image = post.find('meta', property="og:image")['content']
                site_name = post.find('meta', property="og:site_name")['content']
                timeStamp = datetime.now() 
                
                delta = timeStamp - createdAt
                if delta.days <= 0:
                    if my_db.isNewItem('title', title):
                        db_data = { 'title': title, 'desc': desc, 'url': href_full, 'image': image, 'siteName':site_name,
                                            'createdBy': createdBy, 'createdAt': createdAt, 'timeStamp': timeStamp }
                        my_db.insertTable(db_data)          
            else:
                print('error: ', res.status_code) 
    my_db.displayHTML()  
    my_db.close()
else:
    print('error: ', response.status_code)